Name: Yan Chun Shan, Solomon SID: 53660593

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
import IPython.utils.warn as warn
random.seed(100)
import json
import csv
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
import time

/Library/Python/2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Library/Python/2.7/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


Load heloper functions

In [2]:
def load_cooking(fname):
    # load the cooking dataset
    # returns dictionary with data, target (cuisine type), item ids
    
    fp = open(fname, "r")
    dataj = json.load(fp)
    fp.close()
    
    data = []
    ids  = []
    target = []
    for o in dataj:
        if o.has_key('cuisine'):
            target.append(o['cuisine'])
        ids.append(o['id'])
        data.append(o['ingredients'])

    out = {"data": data, "id":ids}
    if len(target) > 0:
        out["target"] =  target
    return out


# write a kaggle submission file for "whats cooking"
def write_csv_kaggle_cooking(fname, ids, target):
    # header
    tmp = [['id', 'cuisine']]
    
    for i in range(len(ids)):
        # add a row (id and class prediction)
        tmp.append([ids[i], target[i]])
        
    # write CSV file
    f = open(fname, 'wb')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

In [17]:
class MLA:
    def __init__(self, train_predictors, train_target, test_predictors, test_target):
        self.train_predictors = train_predictors
        self.train_target = train_target
        self.test_predictors = test_predictors
        self.test_target = test_target
        
    def logicticRegression(self):
        self.timer_start()
        print 'build logictic regression model:'
        params = {'C':[1, 10]}
        self.classifier = LogisticRegression()
        self.classifier = grid_search.GridSearchCV(self.classifier, params)
        self.classifier.fit(self.train_predictors,self.train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
        
        
    def svm(self):
        self.timer_start()
#         Cs = logspace(0.1, 0.9, endpoint=False)
#         avgscores = empty(Cs.shape)
#         avgscores = empty(Cs.shape)
#         for i,C in enumerate(Cs):
#             # create the SVM classifier
#             self.classifier = svm.SVC(kernel='linear', C=C)
#             # calculate the scores for this classifier
#             myscore = cross_validation.cross_val_score(self.classifier, train_predictors, train_target, cv=5)
#             # record the average score
#             avgscores[i] = mean(myscore)
#         # pick the best C
#         besti = argmax(avgscores)
#         bestC = Cs[besti]
#         print "bestC =", bestC
        
        print 'build svm classifier:'
        self.classifier = svm.LinearSVC(C=0.8, penalty="l2", dual=False)
        self.classifier.fit(self.train_predictors,self.train_target).score(train_predictors, train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
        
    def decision_tree(self):
        self.timer_start()
        print 'build decision tree model:'
        self.classifier = tree.DecisionTreeClassifier()
        self.classifier.fit(self.train_predictors,self.train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
    
    def naive_bayes_gaussian(self):
        self.timer_start()
        print 'build naive bayes classifier:'
        self.classifier = naive_bayes.GaussianNB()
        self.classifier.fit(self.train_predictors,self.train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
        
        
    def naive_bayes_multinomial(self):
        self.timer_start()
        print 'build naive bayesmultinomial classifier:'
        self.classifier = naive_bayes.MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
        self.classifier.fit(self.train_predictors,self.train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
        
    def naive_bayes_bernoulli(self):
        self.timer_start()
        print 'build naive bayes bernoulli classifier:'
        self.classifier = naive_bayes.BernoulliNB(alpha=1.0, class_prior=None, fit_prior=True)
        self.classifier.fit(self.train_predictors,self.train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
        
    def randomForest(self):
        self.timer_start()
        print 'build random forest classifier:'
        self.classifier = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)
        self.classifier.fit(self.train_predictors,self.train_target)
        self.predictions = self.classifier.predict(test_predictors)
        self.timer_end()
        self.cal_acc()
    
    def timer_start(self):     
        self.start = time.time()
    
    def timer_end(self):     
        self.end = time.time()
        print "Time: ", self.end - self.start    
        
    def cal_acc(self):
        # calculate accuracy
        Ncorrect = sum(self.test_target == self.predictions)
        acc      = mean(self.test_target == self.predictions)
        print "number correct =", Ncorrect
        print "test accuracy =", acc
    
    def predict(self, predictors):
        return self.classifier.predict(predictors)
        

In [4]:
# Load both train and test data from source 
traindata = load_cooking("train.json")
testdata  = load_cooking("test.json")

In [5]:
# Clean ingredient data
traindata['ingredient_plain'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in traindata['data']]
testdata['ingredient_plain'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in testdata['data']] 


In [6]:
# Create frequency term matrix
vectorizer = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)

traindata_sparse_matrix = vectorizer.fit_transform(traindata['ingredient_plain']).todense()
testdata_sparse_matrix = vectorizer.transform(testdata['ingredient_plain'])

In [7]:
# Partition data into training set and test set
train_predictors, test_predictors, train_target, test_target = \
  cross_validation.train_test_split(traindata_sparse_matrix, traindata['target'], 
  train_size=0.6, test_size=0.4, random_state=4487)

In [18]:
# estimate the accuracy
mla = MLA(train_predictors, train_target, test_predictors, test_target)


In [9]:
mla.logicticRegression()

build logictic regression model:
Time:  38.7301559448
number correct = 12501
test accuracy = 0.785732243872


In [19]:
mla.svm()

build svm classifier:
Time:  4.1739449501
number correct = 12484
test accuracy = 0.784663733501


In [20]:
mla.decision_tree()

build decision tree model:
Time:  45.7204201221
number correct = 9646
test accuracy = 0.606285355123


In [21]:
mla.naive_bayes_gaussian()

build naive bayes classifier:
Time:  29.3554518223
number correct = 4187
test accuracy = 0.263167818982


In [22]:
mla.naive_bayes_multinomial()

build naive bayesmultinomial classifier:
Time:  0.758901119232
number correct = 10514
test accuracy = 0.660842237586


In [23]:
mla.naive_bayes_bernoulli()

build naive bayes bernoulli classifier:
Time:  3.3575758934
number correct = 11354
test accuracy = 0.713639220616


In [24]:
mla.randomForest()

build random forest classifier:
Time:  9.34719395638
number correct = 10949
test accuracy = 0.68818353237


In [25]:
mla.logicticRegression()
predictions = mla.predict(testdata_sparse_matrix)

build logictic regression model:
Time:  44.2182929516
number correct = 12501
test accuracy = 0.785732243872


In [26]:
# Create submission file
write_csv_kaggle_cooking("submission.csv", testdata['id'], predictions)